# vina_demo

## Colab Setup

1. Check the runtime instance
2. Choose whether to use Google Drive (persistence) or temp `/content`
3. Install required Python packages
4. Get course files/data into the runtime
5. Mount Google Drive for persistent storage


In [3]:
# @title 1) Check the runtime
import sys, platform, os, textwrap
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Working dir:", os.getcwd())

!arch

Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Working dir: /content
x86_64


In [4]:
# @title 2) Choose whether to use Google Drive (persistence)
#
# If you want your edits to persist across sessions, use **Drive**.
# If you're just running a quick exercise, you can skip Drive and use the temporary Colab filesystem (`/content`).

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# @title Create directories `~/structbio_course/data` and `~/structbio_course/outputs`

import os
from pathlib import Path

# Change this folder name once; everything else uses it.
COURSE_DIR_NAME = "structbio_course"  # you can rename for your course

USE_DRIVE = True

if "drive" in str(Path("/content/drive")) and os.path.exists("/content/drive") and USE_DRIVE:
    ROOT = Path("/content/drive/MyDrive") / COURSE_DIR_NAME
else:
    ROOT = Path("/content") / COURSE_DIR_NAME

ROOT.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / "data"
(DATA_DIR).mkdir(exist_ok=True)

OUTPUTS_DIR = ROOT / "outputs"
(OUTPUTS_DIR).mkdir(exist_ok=True)

print("ROOT:", ROOT)
print("Use Path object DATA_DIR to access:", DATA_DIR)
print("Use Path object OUTPUTS_DIR to access:", OUTPUTS_DIR)


ROOT: /content/drive/MyDrive/structbio_course
Use Path object DATA_DIR to access: /content/drive/MyDrive/structbio_course/data
Use Path object OUTPUTS_DIR to access: /content/drive/MyDrive/structbio_course/outputs



*   List item
*   List item



... and unzip the archive


In [8]:

# @title Copy over a zip archive of data files and code needed for the *vina* exercise ... and unzip

import os, pathlib, urllib.request

RAW_URL = "https://raw.githubusercontent.com/vvoelz/chem5412-spring2026/main/data/assignment02.zip"
zip_path = pathlib.Path(DATA_DIR) / "assignment02.zip"

urllib.request.urlretrieve(RAW_URL, zip_path)
print("Saved to:", zip_path)

!unzip {zip_path}
!cp -r assignment02 {DATA_DIR}

print(f'Unzipped folder `assignment02` moved to {DATA_DIR}')

Saved to: /content/drive/MyDrive/structbio_course/data/assignment02.zip
Archive:  /content/drive/MyDrive/structbio_course/data/assignment02.zip
   creating: assignment02/
   creating: assignment02/input_files/
  inflating: assignment02/input_files/4ey7.pdb  
  inflating: assignment02/input_files/4ey5.pdb  
  inflating: assignment02/input_files/human_AChE_binders.csv  
  inflating: assignment02/input_files/4ey5_receptor.pdb  
  inflating: assignment02/input_files/4ey7_receptor_prepped.pdb  
   creating: assignment02/input_files/rivastigmine/
  inflating: assignment02/input_files/rivastigmine/rivastigmine_out.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.mol2  
  inflating: assignment02/input_files/rivastigmine/config_singledock  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.smiles  
   creating: assignment02/input_files/huperzine/
  inflating: assignment02/input_files/hup

In [9]:
# @title Download the vina executable

!wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64
!mv vina_1.2.3_linux_x86_64 vina #rename the vina_1.2.3_linux_x86_64 as vina
!chmod +x vina

--2026-02-20 02:20:56--  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/258054635/ae531efa-011f-44eb-86d3-dc1115228edc?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-20T03%3A17%3A07Z&rscd=attachment%3B+filename%3Dvina_1.2.3_linux_x86_64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-02-20T02%3A16%3A59Z&ske=2026-02-20T03%3A17%3A07Z&sks=b&skv=2018-11-09&sig=049UDfFxK2pkukEBwU%2FJiLhYfVdopKa9M1D9msQY7Nc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc3MTU1NDM1NiwibmJmIjoxNzcxNTU0MDU2LCJwYXRoIjoicmVsZWFzZ

## Download packages need for preparing receptor and ligand

**NOTE**: condacolab is needed to install the mgltools package.  Installing this will cause a restart of the kernel. If you get a warning about this, you can ignore it.   Becsue of this, we will later have to reload the DATA_DIR and OUTPUTS_DIR Path

In [10]:
# @title (1) Install Condacolab (< 1min)
%%time

!pip install -q condacolab
import condacolab
condacolab.install()

#@markdown > An automatic restart of the kernel is expected after the execution of this block.
#@markdown >
#@markdown > Stay connected to the same runtime and proceed to the next code block!


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...
CPU times: user 2.47 s, sys: 429 ms, total: 2.9 s
Wall time: 15.5 s


In [1]:
# @title Let's inspect the new conda info
!conda info


     active environment : None
       user config file : /root/.condarc
 populated config files : /usr/local/.condarc
          conda version : 24.11.2
    conda-build version : not installed
         python version : 3.11.11.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=broadwell
                          __conda=24.11.2=0
                          __glibc=2.35=0
                          __linux=6.6.105=0
                          __unix=0=0
       base environment : /usr/local  (writable)
      conda av data dir : /usr/local/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /usr/local/pkgs
                          /root/.conda/pkgs
       envs directories : /usr/local/envs
                          /root/.conda/envs
               platform : linux-64
             user-agent :

In [2]:
# @title Install Python packages (~1 min)
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mgltools
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2026.1.4   |       hbd8a1cb_0         143 KB  conda-forge
    cairo-1.18.4               |       h3394656_0         955 KB  conda-forge
    certifi-2026.1.4           |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94 KB  conda-forge
    font-ttf-source-code-pro-2.038|       h77eed37_0         684 KB  conda-forge
    font-ttf-ubuntu-0.83       |       h77eed37_3         1.5 MB  conda-forge
    fo

In [7]:
# @title Verify that mgltools has been installed

import shutil

# Find the pathname of prepare_receptor4.py
receptor_prep_script = shutil.which('prepare_receptor4.py')
print(f'Path to prepare_receptor4.py: {receptor_prep_script}')

# Find the pathname of prepare_ligand4.py
ligand_prep_script = shutil.which('prepare_ligand4.py')
print(f'Path to prepare_ligand4.py: {ligand_prep_script}')

# Check if python2 is available
python2_path = shutil.which('python2')
print(f'Path to python2: {python2_path}')

#@markdown > You should see that the executables `prepare_receptor4.py` and `prepare_ligand4.py` are available in your path
#@markdown >
#@markdown > You should also have Python2 (!) available to run these scripts

Path to prepare_receptor4.py: /usr/local/bin/prepare_receptor4.py
Path to prepare_ligand4.py: /usr/local/bin/prepare_ligand4.py
Path to python2: /usr/local/bin/python2


In [9]:
# @title Reinstantiate Path objects and create INPUT_FILES = DATA_DIR / `assignment02/input_files`

import os
from pathlib import Path

# Change this folder name once; everything else uses it.
COURSE_DIR_NAME = "structbio_course"  # you can rename for your course

USE_DRIVE = True

if "drive" in str(Path("/content/drive")) and os.path.exists("/content/drive") and USE_DRIVE:
    ROOT = Path("/content/drive/MyDrive") / COURSE_DIR_NAME
else:
    ROOT = Path("/content") / COURSE_DIR_NAME

ROOT.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / "data"
(DATA_DIR).mkdir(exist_ok=True)

OUTPUTS_DIR = ROOT / "outputs"
(OUTPUTS_DIR).mkdir(exist_ok=True)

INPUT_FILES = DATA_DIR / "assignment02/input_files"

print("ROOT:", ROOT)
print("Use Path object DATA_DIR to access:", DATA_DIR)
print("Use Path object OUTPUTS_DIR to access:", OUTPUTS_DIR)
print()
print("Use Path object INPUT_FILES to access:", INPUT_FILES)



ROOT: /content/drive/MyDrive/structbio_course
Use Path object DATA_DIR to access: /content/drive/MyDrive/structbio_course/data
Use Path object OUTPUTS_DIR to access: /content/drive/MyDrive/structbio_course/outputs

Use Path object INPUT_FILES to access: /content/drive/MyDrive/structbio_course/data/assignment02/input_files


# Preparation of receptors and ligands for docking

## Receptors

Here, we will use the [mgltools](https://ccsb.scripps.edu/mgltools/) to prepare PDBQT files for all three human AChE receptors.   I have already cleaned up these files and prepped them fo docking:
* **4ey7_receptor_prepped.pdb** - receptor structure of AChE from AChE-donepezil complex
* **4ey5_receptor_prepped.pdb** - receptor structure of AChE from AChE-huperzine A complex
* **4ey6_receptor_prepped.pdb** - receptor structure of AChE from AChE-galantamine complex

## Receptors

We will also prepare PDBQT files for *five* AChE inhibitors:

| Ligand        | SMILES                                                                 | IC50 (nM) |
|---------------|------------------------------------------------------------------------|-----------|
| donepezil     | COC1=C(C=C2C(=C1)CC(C2=O)CC3CCN(CC3)CC4=CC=CC=C4)OC                   | 8.00      |
| huperzine A   | CC=C1C2CC3=C(C1(CC(=C2)C)N)C=CC(=O)N3                                  | 17.0      |
| galantamine   | CN1CCC23C=CC(CC2OC4=C(C=CC(=C34)C1)OC)O                                | 2010      |
| rivastigmine  | CCN(C)C(=O)OC1=CC=CC(=C1)C(C)N(C)C                                     | 32100     |
| AC6           | ClC1=C(NC=2C=CC(Cl)=CC12)C(=O)NC3=NC(=CS3)                             | 280       |



In [12]:
# @title Create PDBQT files for receptors (~12 s)

import os
from pathlib import Path

receptor_codes = ['4ey7', '4ey5', '4ey6']

for rec_code in receptor_codes:

    receptor_prepped_pdb = INPUT_FILES / f'{rec_code}_receptor_prepped.pdb'
    receptor_pdbqt = INPUT_FILES / f'receptor_{rec_code}.pdbqt'
    !{python2_path} {receptor_prep_script} -r {receptor_prepped_pdb} -o {receptor_pdbqt} -U nphs_lps -v


set verbose to  True
read  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/4ey7_receptor_prepped.pdb
setting up RPO with mode= automatic and outputfilename=  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/receptor_4ey7.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide
set verbose to  True
read  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/4ey5_receptor_prepped.pdb
setting up RPO with mode= automatic and outputfilename=  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/receptor_4ey5.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide
set verbose to  True
read  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/4ey6_receptor_prepped.pdb
setting up RPO with mode= automatic and outputfilename=  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/receptor_4ey6.

In [14]:
# @title Create PDBQT files for the ligands (~3 s)

import os, sys
import pandas as pd

df = pd.read_csv(INPUT_FILES / 'human_AChE_binders.csv')
"""
ligand,SMILES,IC50(nM)
donepezil,COC1=C(C=C2C(=C1)CC(C2=O)CC3CCN(CC3)CC4=CC=CC=C4)OC,8.00
huperzine,CC=C1C2CC3=C(C1(CC(=C2)C)N)C=CC(=O)N3,17.0
galantamine,CN1CCC23C=CC(CC2OC4=C(C=CC(=C34)C1)OC)O,2010
rivastigmine,32100
AC6,ClC1=C(NC=2C=CC(Cl)=CC12)C(=O)NC3=NC(=CS3)C4CCNCC4,280
"""

def run_cmd(cmd, testing=False):
    """Run a shell command on the UNIX command line.

    OPTIONS
    testing          If True, just print the cmd. Default: False
    """

    print('>>', cmd)
    if not testing:
        os.system(cmd)

# Create a series of directories containing each ligand
ligand_list = []
ligand_list += [str(df.loc[index,'ligand']) for index, row in df.iterrows()]

smiles_list = []
smiles_list += [str(df.loc[index,'SMILES']) for index, row in df.iterrows()]

for i in range(len(smiles_list)):

    ligand = ligand_list[i]
    smiles = smiles_list[i]
    print(f"{ligand}, {smiles} ")

    ligdir = INPUT_FILES / ligand
    if not os.path.exists(ligdir):
        os.mkdir(ligdir)

    # Write the ligand to file
    smilesfile = os.path.join(ligdir, f'{ligand}.smiles')
    fout = open(smilesfile, 'w')
    fout.write(smiles)
    fout.close()
    print(f'Wrote: {smilesfile}')

    # Use OpenBabel to convert the SMILES into a 3D MOL2 format and
    # perform a weighted rotor search for lowest energy conformer
    mol2file = smilesfile.replace('.smiles','.mol2')

    os.system(f'obabel {smilesfile} -O {mol2file} --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF')
    print(f'  ...  {mol2file}')

    # Then, prepare ligands for docking using the Autodock script
    pdbqtfile = smilesfile.replace('.smiles','.pdbqt')

    ### ... for some reason the script does not work with files on other directories
    local_mol2file = os.path.basename(mol2file)
    cmd = f'cp {mol2file} {local_mol2file}'   # make a local copy of the mol2file
    run_cmd(cmd)  #, testing=True)

    local_pdbqtfile = os.path.basename(pdbqtfile)
    cmd = f'python2 {ligand_prep_script} -l {local_mol2file} -o {local_pdbqtfile} -U nphs_lps -v'
    run_cmd(cmd)  #, testing=True)
    print(f'  ...  {pdbqtfile}')

    # Cleanup!
    cmd = f'mv {local_pdbqtfile} {pdbqtfile}' # move the local outputfile to the ligand directory
    run_cmd(cmd)  #, testing=True)

    cmd = f'rm {local_mol2file}' # remove the local mol2file
    run_cmd(cmd)  #, testing=True)

donepezil, COC1=C(C=C2C(=C1)CC(C2=O)CC3CCN(CC3)CC4=CC=CC=C4)OC 
Wrote: /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/donepezil.smiles
  ...  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/donepezil.mol2
>> cp /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/donepezil.mol2 donepezil.mol2
>> python2 /usr/local/bin/prepare_ligand4.py -l donepezil.mol2 -o donepezil.pdbqt -U nphs_lps -v
  ...  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/donepezil.pdbqt
>> mv donepezil.pdbqt /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/donepezil.pdbqt
>> rm donepezil.mol2
huperzine, CC=C1C2CC3=C(C1(CC(=C2)C)N)C=CC(=O)N3 
Wrote: /content/drive/MyDrive/structbio_course/data/assignment02/input_files/huperzine/huperzine.smiles
  ...  /content/drive/MyDrive/structbio_course/data/assignment02/input_files/huperzine/huperzine.mol2
>> cp /content/driv

In [29]:
# @title Dock all five ligands to all three receptors (~9 min)

# @markdown ---
# @markdown **Note:** This code is inefficient because we're running a new
# @markdown vina job for every ligand(!).  There is a lot of overhead in loading
# @markdown a grid and keeping it in memory and once loaded, many ligands are
# @markdown usually screened. For our purposes here, we require only
# @markdown 15 total calculations, so we can afford some inefficiencies.

import os, sys
from shutil import copyfile

# Receptors
receptor_codes = ['4ey7', '4ey5', '4ey6']

# Ligand directories
ligdirs = ['donepezil', 'galantamine', 'huperzine', 'rivastigmine', 'AC6']

for rec_code in receptor_codes:

    if rec_code == '4ey7':
        # NOTE: the mean xyz coords for donezepil in 4ey7_chainA.pdb is -14.1,-43.8,27.7
        center_x, center_y, center_z = -14.1, -43.8, 27.7   # Angstrom
        size_x, size_y, size_z = 30, 30, 30
    if rec_code == '4ey5':
        # NOTE: the mean xyz coords for donezepil in 4ey5_chainA.pdb is -9.453	-43.660	31.879
        center_x, center_y, center_z = -9.453, -43.66, 31.88   # Angstrom
        size_x, size_y, size_z = 30, 30, 30
    if rec_code == '4ey6':
        # NOTE: the mean xyz coords for donezepil in 4ey6_chainA.pdb is -9.942	-43.488	30.291
        center_x, center_y, center_z = -9.94, -43.49,	30.29   # Angstrom
        size_x, size_y, size_z = 30, 30, 30

    for ligand in ligdirs:

        receptor_pdbqt = INPUT_FILES / f'receptor_{rec_code}.pdbqt'

        LIG_DIR = INPUT_FILES / ligand
        ligand_pdbqt  = LIG_DIR / f'{ligand}.pdbqt'
        config_file = LIG_DIR / f"config_singledock_{rec_code}"

        OUTFILE = OUTPUTS_DIR / f'docking_{rec_code}_{ligand}.pdbqt'
        LOGFILE = OUTPUTS_DIR / f'docking_{rec_code}_{ligand}.log'

        with open(config_file, "w") as f:
            f.write(f"""# CONFIGURATION FILE

    # INPUT OPTIONS
    receptor = {receptor_pdbqt}
    ligand = {ligand_pdbqt}
    # flex = [flexible residues in receptor in pdbqt format]

    # SEARCH SPACE CONFIGURATIONS
    # Center of the box (coordinates x, y and z
    center_x = {center_x}
    center_y = {center_y}
    center_z = {center_z}
    # Size of the box (dimensions in x, y and z)
    size_x = {size_x}
    size_y = {size_y}
    size_z = {size_z}

    # OUTPUT OPTIONS
    out = {OUTFILE}
    # log = {LOGFILE}

    # OTHER OPTIONS
    # cpu = [value] # more cpus reduces the computation time
    # exhaustiveness = [value] # search time for finding the global minimum, default is 8
    # num_modes = [value] # maximum number of binding modes to generate, default is 9
    # energy_range = [value] # maximum energy difference between the best binding mode and the worst one displayed (kcal/mol), default is 3
    # seed = [value] # explicit random seed, not required
        """)

        cmd = f'./vina --config {config_file}'
        run_cmd(cmd)   #, testing=True)

        print(f'Wrote: {OUTFILE}')


>> ./vina --config /content/drive/MyDrive/structbio_course/data/assignment02/input_files/donepezil/config_singledock_4ey7
Wrote: /content/drive/MyDrive/structbio_course/outputs/docking_4ey7_donepezil.pdbqt
>> ./vina --config /content/drive/MyDrive/structbio_course/data/assignment02/input_files/galantamine/config_singledock_4ey7
Wrote: /content/drive/MyDrive/structbio_course/outputs/docking_4ey7_galantamine.pdbqt
>> ./vina --config /content/drive/MyDrive/structbio_course/data/assignment02/input_files/huperzine/config_singledock_4ey7
Wrote: /content/drive/MyDrive/structbio_course/outputs/docking_4ey7_huperzine.pdbqt
>> ./vina --config /content/drive/MyDrive/structbio_course/data/assignment02/input_files/rivastigmine/config_singledock_4ey7
Wrote: /content/drive/MyDrive/structbio_course/outputs/docking_4ey7_rivastigmine.pdbqt
>> ./vina --config /content/drive/MyDrive/structbio_course/data/assignment02/input_files/AC6/config_singledock_4ey7
Wrote: /content/drive/MyDrive/structbio_course/out

# Analysis of the docking results

## RMSD-to-xtal pose

To calculate the RMSD-to-xtal each each docked pose, we will use a tool called [DockRMSD](https://zhanggroup.org/DockRMSD/) from the Zhang group at University of Michigan to compute RMSD for molecules with symmetry or permuted atom orders.)

In [30]:
# @title Download DOCKRMSD

!wget https://seq2fun.dcmb.med.umich.edu//DockRMSD/DockRMSD.gz
!gunzip DockRMSD.gz
!chmod +x ./DockRMSD

--2026-02-20 04:04:55--  https://seq2fun.dcmb.med.umich.edu//DockRMSD/DockRMSD.gz
Resolving seq2fun.dcmb.med.umich.edu (seq2fun.dcmb.med.umich.edu)... 141.213.137.249
Connecting to seq2fun.dcmb.med.umich.edu (seq2fun.dcmb.med.umich.edu)|141.213.137.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499138 (487K) [application/x-gzip]
Saving to: ‘DockRMSD.gz’

DockRMSD.gz         100%[===================>] 487.44K  --.-KB/s    in 0.08s   

2026-02-20 04:04:55 (5.68 MB/s) - ‘DockRMSD.gz’ saved [499138/499138]



In [37]:
# @title Convert the receptor-bound and docked poses of donepezil to *.mol2 format, which DockRMSD expects


ligands = ['donepezil', 'huperzine', 'galantamine']
receptor_codes = ['4ey7', '4ey5', '4ey6']

ligand_rec_pairs = list(zip(ligands, receptor_codes))

for pair in ligand_rec_pairs:

    ligand = pair[0]
    rec_code = pair[1]

    if rec_code == '4ey7':
        # convert the 4ey7 donepezil xtal structure pose to mol2
        !cat {INPUT_FILES / '4ey7_chainA.pdb'}  | grep E20 > {INPUT_FILES / 'donepezil_xtal.pdb'}
        XTAL_MOL2 = INPUT_FILES / 'donepezil_xtal.mol2'
        !obabel {INPUT_FILES / 'donepezil_xtal.pdb'}  -O {XTAL_MOL2} --ff GAFF

    elif rec_code == '4ey5':
        # convert the 4ey5 huperzine A xtal structure pose to mol2
        !cat {INPUT_FILES / '4ey5_chainA.pdb'}  | grep HUP > {INPUT_FILES / 'huperzine_xtal.pdb'}
        XTAL_MOL2 = INPUT_FILES / 'huperzine_xtal.mol2'
        !obabel {INPUT_FILES / 'huperzine_xtal.pdb'}  -O {XTAL_MOL2} --ff GAFF

    elif rec_code == '4ey6':
        # convert the 4ey6 galantamine xtal structure pose to mol2
        !cat {INPUT_FILES / '4ey6_chainA.pdb'}  | grep GNT > {INPUT_FILES / 'galantamine_xtal.pdb'}
        XTAL_MOL2 = INPUT_FILES / 'galantamine_xtal.mol2'
        !obabel {INPUT_FILES / 'galantamine_xtal.pdb'}  -O {XTAL_MOL2} --ff GAFF
    else:
        print(f'rec_code = {rec_code} unrecognized!!!')
        break


    DOCKED_POSES_PDBQT = OUTPUTS_DIR / f'docking_{rec_code}_{ligand}.pdbqt'
    DOCKED_POSES_MOL2 = OUTPUTS_DIR / f'docking_{rec_code}_{ligand}.mol2'

    # convert our docking poses to mol2 -- produces numbered output files for each pose
    !obabel {DOCKED_POSES_PDBQT} -O {DOCKED_POSES_MOL2} -m


1 molecule converted
9 molecules converted
9 files output. The first is /content/drive/MyDrive/structbio_course/outputs/docking_4ey7_donepezil1.mol2
######################################################################
# DockRMSD (v1.1): docking pose distance calculation                 #
#                                                                    #
# If you use DockRMSD in your work, please cite:                     #
#                                                                    #
# Bell, E.W., Zhang, Y. DockRMSD: an open-source tool for atom       #
# mapping and RMSD calculation of symmetric molecules through graph  #
# isomorphism. Journal of Cheminformatics, 11:40 (2019).             #
######################################################################

Calculated Docking RMSD: 2.562

Total # of Possible Mappings: 1769472
Optimal mapping (First file -> Second file, * indicates correspondence is not one-to-one):
C  1 -> C  1 
C  2 -> C  3 *
C  3 -> C  5 *
C  4 -

In [39]:
# @title Compile the scores for each docked pose in a CSV file with columns: pose,score
#
# For the **self-docking** results, add an extra right-hand column with the RMSD-to-xtal


import os, sys, subprocess

def run_cmd(cmd, testing=False):
    """Run a shell command on the UNIX command line.

    OPTIONS
    testing          If True, just print the cmd. Default: False
    """

    print('>>', cmd)
    if not testing:
        os.system(cmd)

########################
# ANALYSIS

# Create an OUTPUTS_DIR / 'docking_summaries'
SUMMARY_DIR = OUTPUTS_DIR / 'docking_summaries'

if not os.path.exists(SUMMARY_DIR):
    os.mkdir(SUMMARY_DIR)

ligands = ['donepezil', 'huperzine', 'galantamine', 'rivastigmine', 'AC6']
receptor_codes = ['4ey7', '4ey5', '4ey6']

for rec_code in receptor_codes:
  for ligand in ligands:

      resultname = f'{rec_code}_{ligand}'
      OUTFILE = OUTPUTS_DIR / f'docking_{rec_code}_{ligand}.pdbqt'

      ### get docking scores
      vina_scores = []
      fin = open(OUTFILE, 'r')
      lines = fin.readlines()
      fin.close()

      for line in lines:
        if line.count('REMARK VINA RESULT:') > 0:
          # line looks like 'REMARK VINA RESULT:   -11.391      0.000      0.000'
          score = float(line.split()[3])
          vina_scores.append( score )

      print('vina_scores', vina_scores)

      self_docking = False
      if ligand == 'donepezil' and rec_code == '4ey7':
          self_docking = True
      if ligand == 'huperzine' and rec_code == '4ey5':
          self_docking = True
      if ligand == 'galantamine' and rec_code == '4ey6':
          self_docking = True

      ### for self-docking, also compileRMSD-to-xtal
      if self_docking:

        # To get the rmsd-to-xtal of the docked pose, we need to parse the output of DockRMSD
        posed_mol2files = [OUTPUTS_DIR / f'docking_{rec_code}_{ligand}{i}.mol2' for i in range(1,10)]

        rmsd_values = []
        for posed_mol2file in posed_mol2files:

          # Create a subprocess to run ./DockRMSD and grab the output
          XTAL_MOL2 = INPUT_FILES / f'{ligand}_xtal.mol2'
          output = subprocess.check_output(['./DockRMSD', XTAL_MOL2, posed_mol2file], text=True)

          # parse the RMSD from the line "Calculated Docking RMSD: ##.###\n"
          lines = [line for line in output.split('\n') if line.count("Calculated Docking RMSD:") > 0]
          rmsd = float(lines[0].strip().split()[-1])
          rmsd_values.append(rmsd)

        print('rmsd_values', rmsd_values)

      # PRINT and SAVE the results

      csvfile = SUMMARY_DIR / f'{resultname}.csv'
      print('-------------')
      print()
      print(f'Writing to {csvfile} ...')
      fout = open(csvfile, 'w')

      num_poses = len(vina_scores)
      if self_docking:
          header = 'pose,score (kcal/mol),RMSD-to-xtal (A),'
          print(header)
          fout.write(header+'\n')
          for j in range(num_poses):
              line = f"{j+1},{vina_scores[j]},{rmsd_values[j]}"
              print(line)
              fout.write(line+'\n')
      else:
          header = 'pose,score (kcal/mol)'
          print(header)
          fout.write(header+'\n')
          for j in range(num_poses):
              line = f"{j+1},{vina_scores[j]}"
              print(line)
              fout.write(line+'\n')

      fout.close()

vina_scores [-11.22, -10.396, -10.379, -10.334, -10.179, -10.165, -9.92, -9.649, -9.096]
rmsd_values [2.562, 3.379, 1.735, 3.323, 3.611, 1.541, 4.905, 2.567, 4.46]
-------------

Writing to /content/drive/MyDrive/structbio_course/outputs/docking_summaries/4ey7_donepezil.csv ...
pose,score (kcal/mol),RMSD-to-xtal (A),
1,-11.22,2.562
2,-10.396,3.379
3,-10.379,1.735
4,-10.334,3.323
5,-10.179,3.611
6,-10.165,1.541
7,-9.92,4.905
8,-9.649,2.567
9,-9.096,4.46
vina_scores [-9.918, -9.376, -9.256, -9.194, -8.91, -8.884, -8.654, -8.523, -7.854]
-------------

Writing to /content/drive/MyDrive/structbio_course/outputs/docking_summaries/4ey7_huperzine.csv ...
pose,score (kcal/mol)
1,-9.918
2,-9.376
3,-9.256
4,-9.194
5,-8.91
6,-8.884
7,-8.654
8,-8.523
9,-7.854
vina_scores [-8.991, -7.593, -7.436, -7.351, -7.191, -6.87, -6.844, -6.512, -6.487]
-------------

Writing to /content/drive/MyDrive/structbio_course/outputs/docking_summaries/4ey7_galantamine.csv ...
pose,score (kcal/mol)
1,-8.991
2,-7.593
3

In [ ]:
# @title Load in the experimenta values

import pandas as pd
df = pd.read_csv(INPUT_FILES / 'human_AChE_binders.csv')
df

,ligand,SMILES,IC50(nM)
0,donepezil,COC1=C(C=C2C(=C1)CC(C2=O)CC3CCN(CC3)CC4=CC=CC=...,8.0
1,huperzine,CC=C1C2CC3=C(C1(CC(=C2)C)N)C=CC(=O)N3,17.0
2,galantamine,CN1CCC23C=CC(CC2OC4=C(C=CC(=C34)C1)OC)O,2010.0
3,rivastigmine,CCN(C)C(=O)OC1=CC=CC(=C1)C(C)N(C)C,32100.0
4,AC6,ClC1=C(NC=2C=CC(Cl)=CC12)C(=O)NC3=NC(=CS3)C4CC...,280.0


### Instructions

For each ligand, plot the lowest (best) docking score versus the experimental $
\Delta G = +RT \ln K_d \approx +RT \ln$ IC50.  Assume that $RT$ is 0.59 kcal/mol.